In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from snowflake.snowpark import Session
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from snowflake.snowpark.functions import udf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:

import snowflake.connector
connection_parameters = snowflake.connector.connect(
    account= "on89866.central-india.azure",
    user= "LTIT4857",
    password = "Sargun165$",
    role= "EDP_GEN_ROLE",
    warehouse= "ACC360_DEV_DE_WH",
    database= "ACCOUNT360",
    schema= "ACC360_BI"
)


In [3]:
# Create a cursor object.
cur = connection_parameters.cursor()

# Execute a statement that will generate a result set.
sql = "select * from ACC360_SWIPE_MNTH_AGGR_BY_DU_ACC_NAME"
cur.execute(sql)

# Fetch the result set from the cursor and deliver it as the Pandas DataFrame.
df = cur.fetch_pandas_all()


In [3]:
! pip install "snowflake-connector-python[pandas]" 

In [4]:
df.head(5)

,YEAR,MONTH_NAME,DU,ACCOUNT_NAME,BILLED_STATUS,GRADE,SWIPE_CITY,MP_COUNT,HC
0,2022,SEPTEMBER,DATA PU Pool,,PU Pool,P5,PUNE,1,1512.000000
1,2022,SEPTEMBER,DATA CoE & Support,null,PU Management Support,G-1-3,MUMBAI,1,903.000000
2,2022,SEPTEMBER,LF-Unbilled,JOHNSON & JOHNSON,Billed,P2,MUMBAI,16,903.000000
3,2022,OCTOBER,LF-Unbilled,JOHNSON & JOHNSON,Billed,P2,PUNE,3,1543.000000
4,2022,APRIL,S1,INSURITY,Billed,P3,MUMBAI,1,993.000000


In [5]:
print((df.columns))

Index(['YEAR', 'MONTH_NAME', 'DU', 'ACCOUNT_NAME', 'BILLED_STATUS', 'GRADE',
       'SWIPE_CITY', 'MP_COUNT', 'HC'],
      dtype='object')


In [6]:
result = df.dtypes
df = df.convert_dtypes()
df["HC"] = pd.to_numeric(df["HC"])
df["HC"] = pd.to_numeric(df["HC"])
print(df.dtypes)

YEAR              string
MONTH_NAME        string
DU                string
ACCOUNT_NAME      string
BILLED_STATUS     string
GRADE             string
SWIPE_CITY        string
MP_COUNT           Int64
HC               float64
dtype: object


In [7]:
result = df.dtypes
df = df.convert_dtypes()
df["HC"] = pd.to_numeric(df["HC"])
df["HC"] = pd.to_numeric(df["HC"])
print(df.dtypes)

YEAR             string
MONTH_NAME       string
DU               string
ACCOUNT_NAME     string
BILLED_STATUS    string
GRADE            string
SWIPE_CITY       string
MP_COUNT          Int64
HC                Int64
dtype: object


In [8]:
result = df.dtypes
print(result)
df.head()

YEAR             string
MONTH_NAME       string
DU               string
ACCOUNT_NAME     string
BILLED_STATUS    string
GRADE            string
SWIPE_CITY       string
MP_COUNT          Int64
HC                Int64
dtype: object


,YEAR,MONTH_NAME,DU,ACCOUNT_NAME,BILLED_STATUS,GRADE,SWIPE_CITY,MP_COUNT,HC
0,2022,SEPTEMBER,DATA PU Pool,,PU Pool,P5,PUNE,1,1512
1,2022,SEPTEMBER,DATA CoE & Support,null,PU Management Support,G-1-3,MUMBAI,1,903
2,2022,SEPTEMBER,LF-Unbilled,JOHNSON & JOHNSON,Billed,P2,MUMBAI,16,903
3,2022,OCTOBER,LF-Unbilled,JOHNSON & JOHNSON,Billed,P2,PUNE,3,1543
4,2022,APRIL,S1,INSURITY,Billed,P3,MUMBAI,1,993


In [9]:
AVG = df["MP_COUNT"].mean()
print(AVG)

1.7614014251781474


In [10]:
# DEFINE WHICH COLUMNS ARE NUMERIC AND WHICH ARE CATEGORICAL
categorical = []
numerical = []
for i in df.columns:
    if (df[i].dtypes == "string"):
        categorical.append(i)
    else:
        numerical.append(i)

In [11]:
##Split data into test and train
x = df.drop('MP_COUNT', axis=1)
y = df['MP_COUNT']
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=0
)

In [12]:
##CHECK WHETHER ALL COLUMNS ARE INCLUDED IN THE CATEGORICAL AND NUMERICAL (LEN(CAT)) + ((LEN(NUM)+1) == LEN(DF.COLUMNS)
##POSSIBLE ADDITION, ERROR MESSAGE IF LENCAT+LENNUM != LEN(DF.COLUMNS)
print("CATCOL", len(categorical))
numerical.remove('MP_COUNT')
print("NUMVAR", len(numerical))
print(len(df.columns))
##PRINTING THE TRAIN AND TEST DATA TO CHECK IF IT IS SPLIT CORRECTLY
print(x_train, y_train, x_test, y_test)

##CREATE PIPELINE PREPROCESSOR FOR NUMERICAL TRANSFORMATION WHERE WE STANDARDISE NUMERIC FEATURES
##IF ANY MISSING VALUES, THEY ARE REPLACED BY MEAN
numeric_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
##CREATE PIPELINE PREPROCESSOR FOR CATEGORICAL FEATURES TO BE ENCODED SO THAT IT CAN BE INTERPRETED BY SKLEARN
##IF CATEGORICAL COLUMN IS NAN, IT WILL BE REPLACED WITH DEFAULT '0' CONSTANT, ELSE IT WILL BE ENCODED INTO INT VALUES

categorical_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

##APPLY THE TRANSFORMERS TO THE FEATURES USING ColumnTransformer() WHERE WE PASS A LIST OF TUPLES TO THE PARAMETER OF transformrs
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_trans, numerical),
        ('categorical', categorical_trans, categorical)
    ]
)

CATCOL 7
NUMVAR 1
9
      YEAR MONTH_NAME                  DU            ACCOUNT_NAME  \
143   2022       JULY                  C2                CATALENT   
6267  2022        MAY         ID-Unbilled              ICICI BANK   
7425  2022    OCTOBER         ID-Unbilled              ICICI BANK   
5210  2022       JULY         HH-Unbilled                   CISCO   
5740  2022      APRIL        DATA PU Pool                           
...    ...        ...                 ...                     ...   
4373  2022       JULY                  C1                 L’OREAL   
7891  2022       JULY                  B1                    ABSA   
4859  2022      MARCH  DATA CoE & Support                           
3264  2022      APRIL                  C1              SERVICENOW   
2732  2022     AUGUST         BF-Unbilled  FIDELITY INTERNATIONAL   

              BILLED_STATUS GRADE SWIPE_CITY    HC  
143                  Billed    P1  BANGALORE  1036  
6267                 Billed    P4     MUMBAI 

In [13]:
print(x_test)

      YEAR MONTH_NAME                  DU                  ACCOUNT_NAME  \
7632  2022    OCTOBER         BF-Unbilled                       CITIFOX   
1091  2022    OCTOBER        DATA PU Pool                    ICICI BANK   
5992  2022     AUGUST         MN-Unbilled                        SCANIA   
2063  2022      MARCH                  M1                       CLARIOS   
8304  2022    JANUARY                  S1  BB&T INSURANCE HOLDINGS INC.   
...    ...        ...                 ...                           ...   
8095  2022       JULY         RE-Unbilled      INTERIOR SPECIALISTS INC   
5412  2022       JUNE         HH-Unbilled                         CISCO   
8057  2022       JULY  DATA CoE & Support                                 
2717  2022  SEPTEMBER         MN-Unbilled                          OTIS   
1815  2022    OCTOBER         LF-Unbilled             JOHNSON & JOHNSON   

              BILLED_STATUS GRADE SWIPE_CITY    HC  
7632                 Billed    P4     MUMBAI  

In [16]:
##ADD ESTIMATOR TO A PIPELINE, FOR THIS USE-CASE, A RANDOMFORESTCLASSIFIER IS OPTIMAL SINCE WE NEED TO USE A
##CLASSIFIER AS THE OUTPUT IS CATEGORICAL (ATTRITION=1, ATTRITION=0) AND NOT A CONTINUOUS VALUE SUCH AS
##PRODUCTIVITY VALUE, SALARY, AGE, ETC. IN THE CASE THAT WE NEEDED TO PREDICT CONTINUOUS VALUES WE WOULD USE REGRESSORS

##RFC IS THE CLASSIFIER WE TESTED THAT HAD THE BEST ACCURACY ON THIS DATASET

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators = 100, random_state = 26))
])

##FIT ML MODEL TRAINING DATA INTO PIPELINE
rfcmodel = pipeline.fit(x_train, y_train)


In [17]:
predictions = rfcmodel.predict(x_test)

In [18]:

##CHECK IF ALL FEATURES AND ATTRIBUTES ARE CORRECTLY ENTERED IN THE PIPELINE
from math import floor
print("RFC MODEL \n", rfcmodel)
predictions = rfcmodel.predict(x_test)
predictions = [round(item, 0) for item in predictions]
predictions = [floor(item) for item in predictions]
print(predictions)
print(metrics.mean_squared_error(y_test, predictions))
print(metrics.r2_score(y_test, predictions))


RFC MODEL 
 Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['HC']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('encoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                

In [ ]:
# R2 SCORE SHOULD BE BTN 0 - 1 BUT CAN BE NEG GIVING NEG CORRELATION 
#BEST THING IS R SCORE SHUD BE CLOSER TO 1

#MSE / MEAN OF THE TARTGET COLUMN *100 = 10 -20 % 